In [1]:
# https://github.com/awslabs/autogluon
# AutoGluon is a SOTA AutoML framework
# This kernel showcases AutoGluon with RAPIDS GPU acceleration

!pip install "autogluon.tabular[all]==0.2.1b20210618"

     |████████████████████████████████| 267 kB 6.1 MB/s 
     |████████████████████████████████| 339 kB 30.0 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |████████████████████████████████| 187 kB 18.9 MB/s 
     |████████████████████████████████| 22.1 MB 24.6 MB/s 
     |████████████████████████████████| 831.4 MB 1.7 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.7.0
    Uninstalling torch-1.7.0:
      Successfully uninstalled torch-1.7.0
  Attempting uninstall: autogluon.core
    Found existing installation: autogluon.core 0.1.0b20210219
    Uninstalling autogluon.core-0.1.0b20210219:
      Successfully uninstalled autogluon.core-0.1.0b20210219
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.8.1
    Uninstalling torchvision-0.8.1:
      Successfully uninstalled torchvision-0.8.1
  Attempting uninstall: fastai
    Found existing installation: fastai 2.2.5
    Uninstalling fastai-2.2.5:
      Successf

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.tabular.models.knn.knn_rapids_model import KNNRapidsModel
from autogluon.tabular.models.lr.lr_rapids_model import LinearRapidsModel
train_data = TabularDataset('../input/tabular-playground-series-jun-2021/train.csv')
test_data = TabularDataset('../input/tabular-playground-series-jun-2021/test.csv')

label = 'target'

In [3]:
# Fit end-to-end with raw data in under 1 hour, with one line of code
predictor = TabularPredictor(
    label=label,
    eval_metric='log_loss',
    learner_kwargs={'ignored_columns': ['id']}
).fit(
    train_data,
    presets='best_quality',
    hyperparameters={
        KNNRapidsModel: {},
        LinearRapidsModel: {},
        'RF': {},
        'XGB': {'ag_args_fit': {'num_gpus': 1, 'lambda': 4.694629428470789, 'alpha': 0.035648228502371364, 'colsample_bytree': 0.495285972368525, 'colsample_bynode': 0.19236814741374086, 'colsample_bylevel': 0.6059126048457186, 'subsample': 0.5128969710771288, 'eta': 0.01, 'grow_policy': 'depthwise', 'max_depth': 17, 'min_child_weight': 59, 'max_bin': 287, 'deterministic_histogram': False}},
        'CAT': {'ag_args_fit': {'num_gpus': 1}},
        'GBM': [{}, {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, 'GBMLarge'],
        'NN': {'ag_args_fit': {'num_gpus': 1}},
        'FASTAI': {'ag_args_fit': {'num_gpus': 1}},
    },
    time_limit = 3600*5
)

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
import pandas as pd
submission = test_data[['id']]
test_pred_proba = predictor.predict_proba(test_data)
submission = pd.concat([submission, test_pred_proba], axis=1)
submission.to_csv('submission.csv', index=False)
submission.head()

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,200000,0.058834,0.419202,0.159012,0.025001,0.012345,0.146946,0.020719,0.043727,0.114215
1,200001,0.043976,0.078717,0.061623,0.021215,0.014296,0.267607,0.084005,0.295167,0.133394
2,200002,0.020758,0.028141,0.018081,0.010672,0.006247,0.713492,0.029488,0.124522,0.048598
3,200003,0.046973,0.112522,0.083354,0.030161,0.016786,0.252862,0.077877,0.219655,0.159810
4,200004,0.041382,0.110174,0.077183,0.024059,0.014444,0.299258,0.065574,0.225872,0.142054


In [5]:
predictor.leaderboard(silent=True)

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-1.743031,136.406644,16814.188255,0.045511,65.100243,3,True,18
1,LightGBMXT_BAG_L2,-1.743774,117.921075,12657.580762,11.349547,801.924590,2,True,14
2,CatBoost_BAG_L2,-1.744440,107.611187,13340.786102,1.039660,1485.129930,2,True,16
3,LightGBM_BAG_L2,-1.745553,112.668947,12485.997066,6.097420,630.340894,2,True,13
4,WeightedEnsemble_L2,-1.746259,77.074108,11401.152277,0.070372,100.648043,2,True,11
5,NeuralNetFastAI_BAG_L2,-1.747571,111.297388,13680.722739,4.725861,1825.066567,2,True,12
6,CatBoost_BAG_L1,-1.747589,1.176632,1938.680762,1.176632,1938.680762,1,True,5
7,LightGBMXT_BAG_L1,-1.750056,31.621584,763.370626,31.621584,763.370626,1,True,3
8,LightGBM_BAG_L1,-1.751019,19.387225,517.618479,19.387225,517.618479,1,True,2
9,XGBoost_BAG_L1,-1.751517,8.191251,3158.807298,8.191251,3158.807298,1,True,6
